# CNN Food Classifier - Transfer Learning

Training a food classification model using transfer learning with EfficientNetB4 pre-trained on ImageNet.

**Approach**:
- Freeze entire pre-trained base model
- Train only the custom classifier head
- ImageNet normalization for pre-trained weights compatibility
- Data augmentation for regularization
- Early stopping to prevent overfitting

**Target**: 80%+ top-1 accuracy on Food-101

## 1. Environment Setup

In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from datetime import datetime

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

# Enable mixed precision for GPU memory efficiency
if tf.config.list_physical_devices('GPU'):
    from tensorflow.keras import mixed_precision
    mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision enabled: float16 computation with float32 accumulation")
else:
    print("No GPU detected, training will use CPU")

## 2. Configuration

Optimized for EfficientNetB4 and Food-101 dataset.

In [ ]:
# Model configuration
IMAGE_SIZE = (224, 224)  # Using 224 instead of 380 for faster training with minimal accuracy loss
NUM_CLASSES = 101
BATCH_SIZE = 20  # Optimized for EfficientNetB4 with mixed precision on GPU

# Training configuration
MAX_EPOCHS = 40  # Maximum epochs with early stopping
LEARNING_RATE = 1e-3  # Standard LR for training new classifier from scratch

# Regularization
LABEL_SMOOTHING = 0.1  # Prevents overconfident predictions
DROPOUT_RATE = 0.4     # Higher dropout for larger model

# ImageNet normalization - required for pre-trained models
IMAGENET_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
IMAGENET_STD = np.array([0.229, 0.224, 0.225], dtype=np.float32)

# Callbacks
EARLY_STOPPING_PATIENCE = 8  # Stop if no improvement for 8 epochs
LR_PATIENCE = 3              # Reduce LR if no improvement for 3 epochs
LR_FACTOR = 0.5              # Reduce LR by half when plateau

# Paths: work whether kernel cwd is notebooks/cnn, notebooks, or smartfood root
_cwd = os.getcwd()
if os.path.basename(_cwd) == 'cnn':
    BASE_DIR = os.path.abspath(os.path.join(_cwd, '..', '..'))
elif os.path.basename(_cwd) == 'notebooks':
    BASE_DIR = os.path.abspath(os.path.join(_cwd, '..'))
else:
    BASE_DIR = os.path.abspath(_cwd)
DATA_DIR = os.path.join(BASE_DIR, 'data', 'food-101')
MODEL_DIR = os.path.join(BASE_DIR, 'data', 'models', 'cnn')
MODEL_PATH = os.path.join(MODEL_DIR, 'food_classifier_best.keras')
HISTORY_PATH = os.path.join(MODEL_DIR, 'training_history.json')

os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)

print(f"Model: EfficientNetB4")
print(f"Image size: {IMAGE_SIZE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Max epochs: {MAX_EPOCHS} (with early stopping patience {EARLY_STOPPING_PATIENCE})")
print(f"Learning rate: {LEARNING_RATE}")
print(f"\nEstimated training time: 5-8 hours (with early stopping likely 15-25 epochs)")
print(f"Per epoch: ~12-15 minutes")
print(f"\nData: {DATA_DIR}")
print(f"Model output: {MODEL_PATH}")

## 2b. Prepare train/val/test splits

Creates `train/`, `val/`, and `test/` from Food-101 `images/` and `meta/` (or skips if they exist). Run once; if you see download instructions, get Food-101 and re-run.

In [ ]:
# Check if train/val/test already exist
train_dir = os.path.join(DATA_DIR, 'train')
val_dir = os.path.join(DATA_DIR, 'val')
test_dir = os.path.join(DATA_DIR, 'test')
images_dir = os.path.join(DATA_DIR, 'images')
meta_dir = os.path.join(DATA_DIR, 'meta')

DATA_READY = False

if os.path.isdir(train_dir) and os.path.isdir(val_dir) and os.path.isdir(test_dir):
    print(f"Data ready: {train_dir}, val, test found.")
    DATA_READY = True
elif os.path.isdir(images_dir) and os.path.isfile(os.path.join(meta_dir, 'train.txt')) and os.path.isfile(os.path.join(meta_dir, 'test.txt')):
    print("Creating train/val/test from images/ and meta/...")
    train_txt = os.path.join(meta_dir, 'train.txt')
    test_txt = os.path.join(meta_dir, 'test.txt')
    val_ratio = 0.15  # 15% of training per class for validation

    def read_split(path):
        with open(path) as f:
            lines = [line.strip() for line in f if line.strip()]
        by_class = {}
        for line in lines:
            cls, _ = line.split('/')
            by_class.setdefault(cls, []).append(line)
        return by_class

    train_by_class = read_split(train_txt)
    test_by_class = read_split(test_txt)
    class_names = sorted(train_by_class.keys())

    for split, base in [('train', train_dir), ('val', val_dir), ('test', test_dir)]:
        os.makedirs(base, exist_ok=True)
        for c in class_names:
            os.makedirs(os.path.join(base, c), exist_ok=True)

    import random
    random.seed(42)
    for cls in class_names:
        train_list = train_by_class[cls]
        random.shuffle(train_list)
        n_val = max(1, int(len(train_list) * val_ratio))
        val_list = train_list[:n_val]
        train_list = train_list[n_val:]
        for rel in train_list:
            src = os.path.join(images_dir, rel + '.jpg')
            dst = os.path.join(train_dir, rel + '.jpg')
            if os.path.isfile(src) and not os.path.lexists(dst):
                try:
                    os.symlink(os.path.abspath(src), dst)
                except OSError:
                    import shutil
                    shutil.copy2(src, dst)
        for rel in val_list:
            src = os.path.join(images_dir, rel + '.jpg')
            dst = os.path.join(val_dir, rel + '.jpg')
            if os.path.isfile(src) and not os.path.lexists(dst):
                try:
                    os.symlink(os.path.abspath(src), dst)
                except OSError:
                    import shutil
                    shutil.copy2(src, dst)
        for rel in test_by_class.get(cls, []):
            src = os.path.join(images_dir, rel + '.jpg')
            dst = os.path.join(test_dir, rel + '.jpg')
            if os.path.isfile(src) and not os.path.lexists(dst):
                try:
                    os.symlink(os.path.abspath(src), dst)
                except OSError:
                    import shutil
                    shutil.copy2(src, dst)
    print("train/val/test created.")
    DATA_READY = True
else:
    print("Food-101 data not found. To run training:")
    print("1. Download food-101.tar.gz from: https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/")
    print("2. Extract so you have: data/food-101/images/ (101 class subdirs) and data/food-101/meta/ (train.txt, test.txt)")
    print("3. Re-run this cell; it will create data/food-101/train/, val/, test/ from images and meta.")
    print("Expected paths:", train_dir, val_dir, test_dir)

# Summary of train/val/test sample counts
if DATA_READY:
    def _count_split(d):
        return sum(
            len([f for f in os.listdir(os.path.join(d, c)) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            for c in sorted(os.listdir(d))
            if os.path.isdir(os.path.join(d, c))
        )
    n_train = _count_split(train_dir)
    n_val = _count_split(val_dir)
    n_test = _count_split(test_dir)
    n_classes = len([c for c in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, c))])
    print("Train samples:", n_train, "| Validation:", n_val, "| Test:", n_test, "| Classes:", n_classes)

## 3. Load Dataset

In [ ]:
if not DATA_READY or not os.path.isdir(os.path.join(DATA_DIR, 'train')):
    raise FileNotFoundError(
        "Food-101 train/val/test not found. Run the previous cell (2b. Prepare Food-101 data). "
        "If you have not downloaded Food-101 yet: get food-101.tar.gz from "
        "https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/ and extract so that "
        "data/food-101/images/ and data/food-101/meta/ (train.txt, test.txt) exist, then re-run cell 2b."
    )

print("Loading Food-101 dataset...\n")

train_dataset = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'train'),
    labels='inferred',
    label_mode='categorical',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'val'),
    labels='inferred',
    label_mode='categorical',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'test'),
    labels='inferred',
    label_mode='categorical',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

class_names = train_dataset.class_names
train_batches = tf.data.experimental.cardinality(train_dataset).numpy()
val_batches = tf.data.experimental.cardinality(val_dataset).numpy()
test_batches = tf.data.experimental.cardinality(test_dataset).numpy()

print(f"Dataset loaded: {len(class_names)} classes")
print(f"Training batches: {train_batches} (~{train_batches * BATCH_SIZE} images)")
print(f"Validation batches: {val_batches}")
print(f"Test batches: {test_batches}")

## 4. Data Pipeline

Preprocessing steps:
1. Scale pixels to 0-1 range
2. Apply augmentation (training only)
3. Apply ImageNet normalization

Augmentation is subtle to maintain visual recognizability.

In [ ]:
@tf.function
def preprocess(image, label):
    """Scale and apply ImageNet normalization."""
    image = tf.cast(image, tf.float32) / 255.0
    image = (image - IMAGENET_MEAN) / IMAGENET_STD
    return image, label

@tf.function
def augment(image, label):
    """Subtle augmentation for regularization."""
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    image = tf.keras.layers.RandomRotation(0.03)(image)
    image = tf.keras.layers.RandomZoom(0.05)(image)
    image = tf.clip_by_value(image, -3.0, 3.0)
    return image, label

print("Building data pipelines...")

# Training pipeline: scale -> augment -> normalize -> prefetch
train_dataset = train_dataset.map(
    lambda x, y: (tf.cast(x, tf.float32) / 255.0, y),
    num_parallel_calls=tf.data.AUTOTUNE
)
train_dataset = train_dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.map(
    lambda x, y: ((x - IMAGENET_MEAN) / IMAGENET_STD, y),
    num_parallel_calls=tf.data.AUTOTUNE
)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

# Validation and test: only normalize
val_dataset = val_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

print("Data pipelines ready")

## 5. Model Architecture

Transfer learning with EfficientNetB4:
- Pre-trained base frozen (base.trainable = False)
- Custom classifier head trained from scratch

In [ ]:
print("Building model...\n")

# Load pre-trained EfficientNetB4 without top classification layers
base_model = EfficientNetB4(
    weights='imagenet',
    include_top=False,
    input_shape=(*IMAGE_SIZE, 3)
)

# Freeze entire base model
base_model.trainable = False

# Build model with custom classifier head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(DROPOUT_RATE),
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(DROPOUT_RATE),
    layers.Dense(NUM_CLASSES, activation='softmax', dtype='float32')
])

total_params = model.count_params()
trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])

print(f"Model: EfficientNetB4 + custom classifier")
print(f"Total parameters: {total_params:,}")
print(f"Trainable: {trainable_params:,} (classifier only)")
print(f"Frozen: {total_params - trainable_params:,} (pre-trained base)")

## 6. Callbacks

Training optimization callbacks.

In [ ]:
checkpoint = ModelCheckpoint(
    MODEL_PATH,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=EARLY_STOPPING_PATIENCE,
    mode='max',
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=LR_FACTOR,
    patience=LR_PATIENCE,
    mode='max',
    min_lr=1e-7,
    verbose=1
)

callbacks = [checkpoint, early_stop, reduce_lr]

print("Callbacks configured:")
print(f"  Model checkpoint: saves best val_accuracy")
print(f"  Early stopping: patience {EARLY_STOPPING_PATIENCE} epochs")
print(f"  LR reduction: patience {LR_PATIENCE}, factor {LR_FACTOR}")

## 7. Training

Train only the classifier head with frozen base model.

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING),
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top5_accuracy')]
)

print(f"\n{'='*60}")
print(f"Training Classifier (Base Model Frozen)")
print(f"{'='*60}")
print(f"Max epochs: {MAX_EPOCHS}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Early stopping will trigger after {EARLY_STOPPING_PATIENCE} epochs without improvement")
print(f"\nStarting training...\n")

start_time = datetime.now()

history = model.fit(
    train_dataset,
    epochs=MAX_EPOCHS,
    validation_data=val_dataset,
    callbacks=callbacks,
    verbose=2
)

training_time = datetime.now() - start_time
best_val_acc = max(history.history['val_accuracy'])
epochs_trained = len(history.history['val_accuracy'])

print(f"\n{'='*60}")
print(f"Training completed in {training_time}")
print(f"Epochs trained: {epochs_trained}/{MAX_EPOCHS}")
print(f"Best validation accuracy: {best_val_acc*100:.2f}%")
print(f"{'='*60}")

## 8. Evaluation

Evaluate the best model on the test set.

In [ ]:
model.load_weights(MODEL_PATH)

print(f"\n{'='*60}")
print("Test Set Evaluation")
print(f"{'='*60}\n")

test_results = model.evaluate(test_dataset, verbose=2)

test_loss = test_results[0]
test_accuracy = test_results[1]
test_top5 = test_results[2]

print(f"\n{'='*60}")
print("FINAL RESULTS")
print(f"{'='*60}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Top-1 Accuracy: {test_accuracy*100:.2f}%")
print(f"Test Top-5 Accuracy: {test_top5*100:.2f}%")
print(f"\nTraining time: {training_time}")
print(f"Epochs: {epochs_trained}")
print(f"{'='*60}\n")

if test_accuracy >= 0.80:
    print("Target achieved: 80%+ accuracy")
elif test_accuracy >= 0.75:
    print("Strong performance, close to target")
else:
    print("Below target - consider longer training or hyperparameter adjustment")

## 9. Save Results

In [ ]:
training_history = {
    'history': {k: [float(v) for v in values] for k, values in history.history.items()},
    'test_results': {
        'loss': float(test_loss),
        'accuracy': float(test_accuracy),
        'top5_accuracy': float(test_top5)
    },
    'config': {
        'model': 'EfficientNetB4',
        'image_size': IMAGE_SIZE,
        'batch_size': BATCH_SIZE,
        'epochs_trained': epochs_trained,
        'max_epochs': MAX_EPOCHS,
        'learning_rate': LEARNING_RATE,
        'label_smoothing': LABEL_SMOOTHING,
        'dropout_rate': DROPOUT_RATE,
        'training_time': str(training_time)
    }
}

with open(HISTORY_PATH, 'w') as f:
    json.dump(training_history, f, indent=2)

print(f"Training history saved: {HISTORY_PATH}")
print(f"Best model saved: {MODEL_PATH}")

## 10. TensorFlow.js Conversion

Convert for deployment in Next.js application.

In [ ]:
print(f"\n{'='*60}")
print('Converting to TensorFlow.js...')
print(f"{'='*60}\n")

try:
    import tensorflowjs as tfjs
    
    tfjs_dir = os.path.join(MODEL_DIR, 'tfjs')
    
    tfjs.converters.save_keras_model(
        model,
        tfjs_dir,
        quantization_dtype=None
    )
    
    print(f"TensorFlow.js model saved: {tfjs_dir}")
    print("Ready for Next.js deployment")
    
except ImportError:
    print("tensorflowjs not installed, skipping conversion")
    print("Install: pip install tensorflowjs==3.21.0")
except Exception as e:
    print(f"Conversion failed: {e}")